In [ ]:
# download small dataset from Google Drive:
# https://drive.google.com/uc?id=1EgHdxQ_YxqIQlvvq5iKlCrkEKR6-j0Ib # items_shuffle_1000 - product scraped info

In [2]:
import json
from pprint import pprint

with open('../data/items_shuffle_1000.json') as f:
    data = json.load(f)
    
pprint(data[0])

{'aplus_present': '',
 'asin': 'B06Y3VLDFB',
 'availability_quantity': 6,
 'availability_status': 'Only 6 left in stock - order soon.',
 'average_rating': '',
 'brand': 'Brand: Vhomes Lights',
 'brand_url': 'https://www.amazon.com/s/ref=bl_dp_s_web_0?ie=UTF8&search-alias=aps&field-keywords=Vhomes+Lights',
 'category': 'garden',
 'customization_options': '',
 'fast_track_message': '',
 'fulfilled_by_amazon': '',
 'full_description': 'Solid, Reclaimed Fir Wood In Natural, Sun Bleached '
                     'Finish With Light Antiquing Glaze.',
 'images': ['https://m.media-amazon.com/images/I/51Y1zY1ZdZL.jpg',
            'https://m.media-amazon.com/images/I/41fbRZ6LKlL.jpg',
            'https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.gif'],
 'list_price': '',
 'name': 'Vhomes Lights Reclaimed Wood Console Table The Genessis Collection '
         'Sofa-Tables',
 'page': 378,
 'pricing': '$877.80',
 'product_category': 'Home & Kitchen › Furniture › L

In [4]:
from langchain.agents import tool

# Placeholder retriever function
    # terrible efficiency, should only use on `small` dataset with 1000 items
@tool
def search_product(query: str) -> str:
    """Search product in database by name"""
    return [p for p in data if query.lower() in p['name'].lower()]

In [5]:
search_product.invoke('counter')

[{'name': '3-Piece Dining Table Set, Farmhouse Counter Height Dining Table Set, Wood Dining Table Set with Drop Leaf Table/2 Cross Back Padded Chairs/One Shelf',
  'product_information': {'Item Weight': '\u200e85 pounds',
   'Product Dimensions': '\u200e42.3 x 24 x 35.3 inches',
   'Country of Origin': '\u200eVietnam',
   'Item model number': '\u200eDining Table Set',
   'Is Discontinued By Manufacturer': '\u200eNo',
   'Weight': '\u200e85 Pounds',
   'ASIN': 'B09B7G7P58',
   'Customer Reviews': {'ratings_count': None, 'stars': '4.0 out of 5 stars'},
   'Best Sellers Rank': ['#431,117 in Home & Kitchen (See Top 100 in Home & Kitchen)   #321 in Kitchen & Dining Room Sets'],
   'Date First Available': 'July 27, 2021'},
  'brand': 'Brand: VOGU',
  'brand_url': 'https://www.amazon.com/s/ref=bl_dp_s_web_0?ie=UTF8&search-alias=aps&field-keywords=VOGU',
  'full_description': '♬ Offer 2 great dining chairs and an attractive kitchen table,this refined dinette set will reinvent your dining room 

In [8]:
# API keys
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

sys_prompt = """
You are an online shopping assistant. Your job is to help users find products, provide product details, and answer questions.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sys_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_product], 
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=[search_product], 
    verbose=True
)

In [14]:
agent_executor.invoke({"input": "I want to buy a counter",})



> Entering new AgentExecutor chain...

Invoking: `search_product` with `{'query': 'counter'}`


[{'name': '3-Piece Dining Table Set, Farmhouse Counter Height Dining Table Set, Wood Dining Table Set with Drop Leaf Table/2 Cross Back Padded Chairs/One Shelf', 'product_information': {'Item Weight': '\u200e85 pounds', 'Product Dimensions': '\u200e42.3 x 24 x 35.3 inches', 'Country of Origin': '\u200eVietnam', 'Item model number': '\u200eDining Table Set', 'Is Discontinued By Manufacturer': '\u200eNo', 'Weight': '\u200e85 Pounds', 'ASIN': 'B09B7G7P58', 'Customer Reviews': {'ratings_count': None, 'stars': '4.0 out of 5 stars'}, 'Best Sellers Rank': ['#431,117 in Home & Kitchen (See Top 100 in Home & Kitchen)   #321 in Kitchen & Dining Room Sets'], 'Date First Available': 'July 27, 2021'}, 'brand': 'Brand: VOGU', 'brand_url': 'https://www.amazon.com/s/ref=bl_dp_s_web_0?ie=UTF8&search-alias=aps&field-keywords=VOGU', 'full_description': '♬ Offer 2 great dining chairs and an attractive kitchen

{'input': 'I want to buy a counter',
 'output': 'I found a few options for counters:\n\n1. **3-Piece Dining Table Set, Farmhouse Counter Height Dining Table Set**\n   - **Brand:** VOGU\n   - **Price:** $319.00\n   - **Description:** This dining table set includes 2 chairs and an extendable drop leaf table. It features a farmhouse design with a shelf for extra storage. The table is made of MDF and acacia veneer with rubber wood legs, while the chairs have linen fabric cushions.\n   - **Rating:** 4.0 out of 5 stars\n   - **Availability:** In Stock.\n   - **[Check it out on Amazon](https://www.amazon.com/s/ref=bl_dp_s_web_0?ie=UTF8&search-alias=aps&field-keywords=VOGU)**\n   - ![Image](https://m.media-amazon.com/images/I/51oBcRkCySL.jpg)\n\n2. **Cabilock Metal Vanity Tray**\n   - **Brand:** Cabilock\n   - **Price:** $13.99\n   - **Description:** This decorative tray can be used for jewelry, trinkets, or as a food serving plate. It is made of iron and comes in a black color. Perfect for or

Next steps:
- Pass in chat history via `agent_executor.invoke({"input": "some text", "chat_history": []})`
- Implement database & retriever (price filters, category, etc.)
- User-defined section in system prompt (vulnerable to injection attack??)
- Use Vision model to take picture input e.g. clothing style reference pic OR dedicated Vision node